# Imports

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import wrangle

from os.path import exists

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from importlib import reload

import warnings
#warnings.filterwarnings("ignore")


# Acquire

In [3]:
raw = pd.read_csv('oct22pub.csv')

In [10]:
df = df[second_cut]

In [17]:
# create function to flatten race in less categories
def flatten_race(val):
    if val == 1:
        val = 'white'
    elif val == 2:
        val = 'black'
    elif val == 3:
        val = 'AI/NA'
    elif val == 4:
        val = 'asian'
    elif val == 5:
        val = 'HI/PI'
    elif 5 < val < 10:
        val = 'mixed_white'
    elif 10 <= val < 27 :
        val = 'mixed_other'
    return val

df['ptdtrace'] = df['ptdtrace'].apply(flatten_race)

#artifact from trying to combine with Hispanic status
# df[['ptdtrace','pehspnon']].sort_values('pehspnon').value_counts()

# Prep: Part IV 
* Goal:
    - Create functions that will handle sections of prep 
    - Big wrangle Function --> prep_aug()
    - don't forget to add docstring placeholders

In [32]:
def acquire_aug():
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    df = pd.read_csv('aug22pub.csv')

    df.columns = df.columns.str.lower()
    
    return df

# create function to flatten race in less categories
def flatten_race(val):
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''
    if val == 1:
        val = 'white'
    elif val == 2:
        val = 'black'
    elif val == 3:
        val = 'AI/NA'
    elif val == 4:
        val = 'asian'
    elif val == 5:
        val = 'HI/PI'
    elif 5 < val < 10:
        val = 'mixed_white'
    elif 10 < 27 :
        val = 'mixed_other'
    return val


def prep_values(df):
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    #decided to dropna's cleared up problem with target variable
    df = df.dropna()

    # fixing peafnow  --> get rid of peafnow
    df[df.peafnow == 1].loc[:,'peafever'] = 1

    #work done to fix usual_hours_worked with manual imputation 
    #mean hours worked for those less that work less than 35
    more_than_35 = round(df[df.pehruslt > 35].pehruslt.mean())
    #mean hours worked for those less that work less than 35
    less_than_35 = round(df[(df.pehruslt > 0) & (df.pehruslt < 35)].pehruslt.mean())
    df[['pehrftpt', 'pehruslt']].value_counts()

    df[df.pehrftpt == 1].loc[:,'pehruslt'] = more_than_35
    df[df.pehrftpt == 2].loc[:,'pehruslt'] = less_than_35

    # remove responses from people not in the labor force by means other that discouragement
    df = df[df.prempnot != 4]
    # remove responses from unqualified respondents (children and active armed forces members)
    df = df[df.prempnot != (-1)]
    # binary recode of premmpnot to include repsondents discourage from workforce participation as unemployed
    df.prempnot = np.where(df.prempnot == 1, 1, 0)

    return df


def prep_columns(df): 
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    df['ptdtrace'] = df['ptdtrace'].apply(flatten_race)

    final_list = ['hehousut', 'hefaminc', 'hrnumhou', 'hrhtype','hubus',\
    'gediv', 'gestfips', 'gtmetsta', 'gtcbsasz', 'prtage', 'pesex', \
    'pemaritl', 'peafever', 'peeduca', 'ptdtrace', 'pehspnon', 'penatvty', 'pemntvty',
    'pefntvty', 'prcitshp', 'pubus1', 'pehruslt', 'pelkavl', 'pedwlko', 'pedwwk', 'pejhwant', 'prempnot',\
    'prmjind1', 'prmjocc1', 'peschenr', 'prchld', 'pecert1']

    df = df[final_list]

    df = df.rename(columns={'hehousut': 'housing_type',
        'hefaminc': 'family_income',
        'hrnumhou': 'household_num',
        'hrhtype': 'household_type',
        'hubus': 'own_bus_or_farm',
        'gediv': 'country_region',
        'gestfips': 'state',
        'gtmetsta': 'metropolitan',
        'gtcbsasz': 'metro_area_size',
        'prtage': 'age',
        'pesex': 'is_male',
        'pemaritl': 'marital_status',
        'peafever': 'veteran',
        'peeduca': 'education',
        'ptdtrace': 'race',
        'pehspnon': 'hispanic_or_non',
        'penatvty': 'birth_country',
        'pemntvty': 'mother_birth_country',
        'pefntvty': 'father_birth_country',
        'prcitshp': 'citizenship',
        'pubus1': 'upaid_work_last_week',
        'pehruslt': 'usual_hours_worked',
        'prempnot': 'employed',
        'prmjind1': 'industry',
        'prmjocc1': 'occupation',
        'peschenr': 'enrolled_in_school',
        'prchld': 'children_in_household',
        'pecert1': 'professional_certification'})

    #fixing types on categorical columns
    categorical_cols = ['housing_type','family_income','household_type',
                        'country_region','state','metropolitan','metro_area_size',
                        'marital_status','education','race','birth_country',
                        'mother_birth_country','father_birth_country','citizenship',
                        'industry','occupation']

    binary_cols = ['own_bus_or_farm', 'is_male', 'veteran','hispanic_or_non', 
                'upaid_work_last_week','employed',
                'enrolled_in_school','professional_certification']  

    #for loop to handle assignment as object
    for col in categorical_cols:
        df[col] = (df[col].astype('object'))

    # changed all non-affirmative answers to negative. tips, school, most affected
    for col in binary_cols:
        df[col] = np.where(df[col] == 1, 1, 0)                                 

    return df

In [33]:
def prep_aug():
    ''' 
    Purpose:

    ---

    Parameters:

    ---

    Output:
    
    ---
    '''

    df = acquire_aug()

    df = prep_values(df)

    df = prep_columns(df)

    return df

In [34]:
test = prep_aug()

/var/folders/x7/9w9mh0fd73zg4jr8l_9v9h840000gn/T/ipykernel_73387/4138682974.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.peafnow == 1].loc[:,'peafever'] = 1
/var/folders/x7/9w9mh0fd73zg4jr8l_9v9h840000gn/T/ipykernel_73387/4138682974.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.pehrftpt == 1].loc[:,'pehruslt'] = more_than_35
/var/folders/x7/9w9mh0fd73zg4jr8l_9v9h840000gn/T/ipykernel_73387/4138682974.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50806 entries, 1 to 127917
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   housing_type                50806 non-null  object 
 1   family_income               50806 non-null  object 
 2   household_num               50806 non-null  int64  
 3   household_type              50806 non-null  object 
 4   own_bus_or_farm             50806 non-null  int64  
 5   country_region              50806 non-null  object 
 6   state                       50806 non-null  object 
 7   metropolitan                50806 non-null  object 
 8   metro_area_size             50806 non-null  object 
 9   age                         50806 non-null  float64
 10  is_male                     50806 non-null  int64  
 11  marital_status              50806 non-null  object 
 12  veteran                     50806 non-null  int64  
 13  education                   50

In [36]:
reload(wrangle)

<module 'wrangle' from '/Users/sinao/coding_projects/oct_2022_unemployment_project/wrangle.py'>

# October Work

# Acquire  

In [104]:
df = pd.read_csv('oct22pub.csv')

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126649 entries, 0 to 126648
Columns: 388 entries, hrhhid2 to pternh2
dtypes: float64(337), int64(51)
memory usage: 374.9 MB


In [106]:
# list to hold columns for masking. 

In [107]:
columns_to_keep = [ 'hehousut',
                    'hrnumhou', 
                    'hefaminc',
                    'hrhtype',
                    'hubus',
                    'gediv',
                    'gestfips',
                    'gtmetsta',
                    'gtcbsasz',
                    'prtage',
                    'pemaritl',
                    'pesex',
                    'peafever',
                    'peeduca',
                    'ptdtrace', 
                    'pehspnon',
                    'penatvty',
                    'pemntvty',
                    'pefntvty', 
                    'prcitshp', 
                    'prinuyer',
                    'prempnot',
                    'prftlf',
                    'prcow1',
                    'prmjind1',
                    'prmjocc1', 
                    'peschenr', 
                    'prnmchld',
                    'pedipged',
                    'peafwhn1',
                    'pecert1',\
                    
        ]

In [108]:
df.columns = df.columns.str.lower()

In [109]:
df = df[columns_to_keep]

In [110]:
df = df.rename(columns={'hehousut': 'housing_type',
        'hrnumhou': 'household_num',
        'hefaminc': 'family_income',
        'hrhtype': 'household_type',
        'hubus': 'own_bus_or_farm',
        'gediv': 'country_region',
        'gestfips': 'state',
        'gtmetsta': 'metropolitan',
        'gtcbsasz': 'metro_area_size',
        'prtage': 'age',
        'pemaritl': 'marital_status',
        'pesex': 'is_male',
        'peafever': 'veteran',
        'peeduca': 'education',
        'ptdtrace': 'race',
        'pehspnon': 'hispanic_or_non',
        'penatvty': 'birth_country',
        'pemntvty': 'mother_birth_country',
        'pefntvty': 'father_birth_country',
        'prcitshp': 'citizenship',        
        'prinuyer': 'immigration_era',
        'prempnot': 'employed',
        'prftlf': 'ft_or_pt',
        'prcow1': 'worker_class',
        'prmjind1': 'industry',
        'prmjocc1': 'occupation',
        'peschenr': 'enrolled_in_school',
        'prnmchld': 'num_children',
        'pedipged': 'ged_or_diploma',
        'peafwhn1': 'service_era',
        'pecert1': 'professional_certification'})

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126649 entries, 0 to 126648
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   housing_type                126649 non-null  int64  
 1   household_num               126649 non-null  int64  
 2   family_income               126649 non-null  int64  
 3   household_type              126649 non-null  int64  
 4   own_bus_or_farm             126649 non-null  int64  
 5   country_region              126649 non-null  int64  
 6   state                       126649 non-null  int64  
 7   metropolitan                126649 non-null  int64  
 8   metro_area_size             126649 non-null  int64  
 9   age                         100839 non-null  float64
 10  marital_status              100839 non-null  float64
 11  is_male                     100839 non-null  float64
 12  veteran                     100839 non-null  float64
 13  education     

# Prepare 

### drop any NILF(not in labor force or with -1)

In [112]:
df.employed.value_counts(0)

 1.0    48086
 4.0    33496
-1.0    17580
 2.0     1589
 3.0       88
Name: employed, dtype: int64

In [113]:
df = df[(df.employed == 1) | (df.employed == 2)].reset_index(drop=True)

### check for null values

In [114]:
df.isna()

,housing_type,household_num,family_income,household_type,own_bus_or_farm,country_region,state,metropolitan,metro_area_size,age,...,ft_or_pt,worker_class,industry,occupation,hourly,enrolled_in_school,num_children,ged_or_diploma,service_era,professional_certification
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49670,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49671,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49672,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49673,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [115]:
df.household_num.value_counts()

2     16209
3      9881
4      9622
1      6358
5      4511
6      1905
7       692
8       278
9       129
10       36
11       35
12       12
13        7
Name: household_num, dtype: int64

### check for -1 values (those are the N/A type responses in the survey). Will try to impute them depeneding on the category

In [116]:
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_values())
    print('---')

housing_type
8         2
12       10
4        12
7        13
3        13
2        17
6       220
5      1556
1     47832
Name: housing_type, dtype: int64
---
household_num
13        7
12       12
11       35
10       36
9       129
8       278
7       692
6      1905
5      4511
1      6358
4      9622
3      9881
2     16209
Name: household_num, dtype: int64
---
family_income
3       246
2       285
5       384
1       426
4       466
6       725
7      1088
8      1251
9      1844
10     2084
11     3248
12     3748
13     5248
14     7342
15     9544
16    11746
Name: family_income, dtype: int64
---
household_type
10        4
9         8
8        16
5        17
2       189
3      3428
7      4805
4      6125
6      6162
1     28921
Name: household_type, dtype: int64
---
own_bus_or_farm
-1        1
-2       41
-3      119
 1     8518
 2    40996
Name: own_bus_or_farm, dtype: int64
---
country_region
6    3267
1    3976
2    4202
4    4825
3    5245
7    5470
8    6184
9    7377
5    

### takeaways
* columns to adjust
    own_bus_or_farm, veteran, unpaid_work_last_week, usual_35_or_more
* to drop, job_past_12_months, worker_class, industry, occupation, hourly, enrolled_in_school, ged_or_diploma, service_era
---

## imputing values and handling -1
* need to handle the -1 values by reading original data dictionary for guidance
* an answer not in the affirmative is treated as a negative for binary classification
    * own_bus_or_farm, veteran, service_era
* delete columns
    * unpaid, usual 35 or more, job past 12 months
* impute (decide on method) --> KNN impute
    * enrolled_in_school

In [117]:
df['own_bus_or_farm'] = df['own_bus_or_farm'].apply(lambda x: x == 1, 1,0)

In [118]:
df['veteran'] = df['veteran'].apply(lambda x: x == 1, 1,0)

In [119]:
df['service_era'] = df.service_era.apply(lambda x: 0 if x == -1 else x)

In [120]:
df.enrolled_in_school = df.enrolled_in_school.apply(lambda x: np.nan if x == -1 else x)

In [121]:
from sklearn.impute import KNNImputer

In [122]:
X = df.enrolled_in_school

In [123]:
imputer = KNNImputer(n_neighbors=1)
df= pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49675 entries, 0 to 49674
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   housing_type                49675 non-null  float64
 1   household_num               49675 non-null  float64
 2   family_income               49675 non-null  float64
 3   household_type              49675 non-null  float64
 4   own_bus_or_farm             49675 non-null  float64
 5   country_region              49675 non-null  float64
 6   state                       49675 non-null  float64
 7   metropolitan                49675 non-null  float64
 8   metro_area_size             49675 non-null  float64
 9   age                         49675 non-null  float64
 10  marital_status              49675 non-null  float64
 11  is_male                     49675 non-null  float64
 12  veteran                     49675 non-null  float64
 13  education                   496

### Takeaways
* Nothing of major importance 
* was difficult understanding how to utilize the KNN imputer with the categorical nature of the column being imputed
---

## Flattening Columns

* Columns to flatten: 
 * housing_type, family income, household_type, marital_status, veteran, education, race, birth country, mother birth country, father birth country, immigration era, service era

## Categorical vs Numerical

In [ ]:
#fixing types on categorical columns
categorical_cols = ['housing_type','family_income','household_type',
                    'country_region','state','metropolitan','metro_area_size',
                    'marital_status','education','race','birth_country',
                    'mother_birth_country','father_birth_country','citizenship',
                    'industry','occupation']

#for loop to handle assignment as object
for col in categorical_cols:
    df[col] = (df[col].astype('object'))

In [ ]:
#fixing binary columns freom -1, 1, 2 --> 0, 1
binary_cols = ['own_bus_or_farm', 'sex', 'veteran','hispanic_or_non', 
                'upaid_work_last_week','could_have_started_job',
                'sought_work_last_12_months', 'worked_last_12_months',
                'intend_to_look_next_12_months','employed', 'usual_ot_tips_commis',
                'enrolled_in_school','professional_certification']


In [ ]:
# changed all non-affirmative answers to negative. tips, school, most affected
for col in binary_cols:
    df[col] = np.where(df[col] == 1, 1, 0)